In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"
out_folder = "/home/mlopez/EXEC/TimeMeansModels/"

In [34]:
shapefile = ("/home/mlopez/EXEC/Shapefiles/DDE_STF_20K_REG_FOR_VUE_S.shp")
poly = gpd.read_file(shapefile)
poly= poly.to_crs('epsg:4326')
#poly

# Select variable

In [38]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

filesSeas45 =  glob.glob(folder+"*rcp45_"+var+"_seasonal.nc")
filesSeas85 =  glob.glob(folder+"*rcp85_"+var+"_seasonal.nc")
#files = glob.glob(folder+"*rcp85_tg_mean_annual.nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
#print(files)
#print(filesSeas)


# FOR RCP 45

In [46]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Getting time windows and percentiles

In [48]:
# Table 
# Mapping data - 30 year means
tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
perc30yavgTable 

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

In [50]:
%time dfperT = perc30yavgTable.drop('realization').to_dataframe().dropna()

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 51.1 s, sys: 4min 17s, total: 5min 8s
Wall time: 4.99 s


## Split by periodos

In [54]:
df45T1980 = dfperT.query("time=='1981-01-01'")
df45T2041= dfperT.query("time=='2041-01-01'")
df45T2071 = dfperT.query("time=='2071-01-01'")

## TODO
- Cut by region
- get the mean by region
- Do the same for rcp84 and seasonal 45 and 85. 
- Put all together into the table

## Seasonal

In [42]:
dsEnsSeas45 = ens.create_ensemble(filesSeas45)

## To split by region

In [55]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [56]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [57]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

# Creating ROWS

In [19]:
col1 = ["Annuel","Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
        "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]

# Creating COLUMNS

In [ ]:
dfTable = pd.DataFrame()
dfTable["Saison"] = col1

In [44]:
#dfTable

## Adding columns for RCP85

In [33]:
dfTable["1981-2010"] = [round(dfReg1T1980.tg_mean_p50.values[0]-273.15, 2), (str(round(dfReg1T1980.tg_mean_p10.values[0]-273.15,2))
                                                            +" - "+str(round(dfReg1T1980.tg_mean_p90.values[0]-273.15,2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dfTable["2041-2070"] = [round(dfReg1T2041.tg_mean_p50.values[0]-273.15, 2), (str(round(dfReg1T2041.tg_mean_p10.values[0]-273.15,2))
                                                            +" - "+str(round(dfReg1T2041.tg_mean_p90.values[0]-273.15,2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dfTable["2071-2100"] = [round(dfReg1T2071.tg_mean_p50.values[0]-273.15, 2), (str(round(dfReg1T2071.tg_mean_p10.values[0]-273.15,2))
                                                            +" - "+str(round(dfReg1T2071.tg_mean_p90.values[0]-273.15,2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dft2 = dfTable.set_index("Saison")

In [29]:
dft2

,1981-2010,2041-2070,2071-2100
Saison,,,
Annuel,2.82903,5.95568,8.71639
Percentiles (10 - 90),2.79 - 3.03,5.31 - 6.9,6.5 - 9.8
Hiver,1-99,1-99,1-99
Percentiles (10 - 90),2,2,2
Printemps,3,3,3
Percentiles (10 - 90),4,4,4
Ete,5,5,5
Percentiles (10 - 90),6,6,6
Automne,7,7,7


In [24]:
dft2.to_csv('dfTable.csv')